In [2]:
#Import libreries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [3]:
# Import CSV

df = pd.read_csv('https://raw.githubusercontent.com/allisonhorst/palmerpenguins/master/inst/extdata/penguins.csv')
df.head()

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,year
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,male,2007
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,female,2007
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,female,2007
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN,2007
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,female,2007


In [8]:
# 1
df.dtypes

species               object
island                object
bill_length_mm       float64
bill_depth_mm        float64
flipper_length_mm    float64
body_mass_g          float64
sex                   object
year                   int64
dtype: object

In [7]:
# 2
(df
 .dtypes
 .value_counts()
)

float64    4
object     3
int64      1
dtype: int64

In [9]:
# 3
print('the dataset has {} rows and {} columns'.format(df.shape[0],df.shape[1]))

the dataset has 344 rows and 8 columns


# Data Validation

1) ¿Qué tipo de dato son las variables del conjunto de datos?

2) ¿Cuántas variables de cada tipo de dato tenemos en el conjunto de datos?

3) ¿Cuántas variables y observaciones tenemos en el conjunto de datos?

4) ¿Existen valores nulos explícitos en el conjunto de datos?

5) De tener observaciones con valores nulos, ¿cuántas tenemos por cada variable?
6) ¿Cuántos valores nulos tenemos en total en el conjunto de datos?
7) ¿Cuál es la proporción de valores nulos por cada variable?
8) ¿Cómo podemos visualizar los valores nulos en todo el conjunto de datos?
9) ¿Cuántas observaciones perdemos si eliminamos los datos faltantes?

In [11]:
# 3
print(''.format(df.isnull().sum().sum()))

19